In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import bezier_curve as bz
import display as d

print("import success")

import success


In [4]:
control_points = np.array([
    [0, 0],
    [19, 19]
], dtype=int)

In [5]:
import graphics as g

screen = g.create_screen((20, 20))

g.draw_points_p(screen, control_points)
g.print_screen(screen)

|                                                          O |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                      

In [6]:
# fig = d.create_figure()

# d.add_control_points(fig, control_points)

# fig.update_layout(title="Control Points")
# fig.show("png")

In [7]:
n_segments = 20
curve = bz.get_curve(n_segments, control_points)
curve = np.rint(curve).astype(int)
print("curve", curve.shape)

curve (21, 2)


In [8]:
screen = g.clear_screen(screen)
g.draw_points_xy(screen, curve)
g.draw_points_p(screen, control_points)
g.print_screen(screen)

|                                                          O |
|                                                       .    |
|                                                    .       |
|                                                 .          |
|                                              .             |
|                                           .                |
|                                        .                   |
|                                     .                      |
|                                  .                         |
|                               .                            |
|                            .                               |
|                         .                                  |
|                      .                                     |
|                   .                                        |
|                .                                           |
|             .                                        

In [9]:
# d.add_curve(fig, curve)

# fig.update_layout(title="Linear Interpolation")
# fig.show("png")

In [10]:
control_points = np.array([
    [0, 19],
    [0, 0],
    [19, 0]
], dtype=int)

In [11]:
screen = g.clear_screen(screen)
#g.draw_points_xy(screen, curve)
g.draw_points_p(screen, control_points)
g.print_screen(screen)

| O                                                          |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                            |
|                                                      

In [12]:
# fig.data = []

# d.add_control_points(fig, control_points)

# fig.update_layout(title="Control Points")
# fig.show("png")

In [13]:
n_segments = 20
curve = bz.get_curve(n_segments, control_points)
curve = np.rint(curve).astype(int)
print("curve", curve.shape)

curve (21, 2)


In [14]:
screen = g.clear_screen(screen)
g.draw_points_xy(screen, curve)
g.draw_points_p(screen, control_points)
g.print_screen(screen)

| O                                                          |
|                                                            |
| .                                                          |
|                                                            |
| .                                                          |
| .                                                          |
|                                                            |
|    .                                                       |
|    .                                                       |
|                                                            |
|       .                                                    |
|       .                                                    |
|          .                                                 |
|             .                                              |
|                .                                           |
|                   .                                  

In [15]:
# d.add_curve(fig, curve)

# fig.update_layout(title="Quadratic Bezier Curve")
# fig.show("png")

In [16]:
# fig.data = []

# def plot_quadratic(figure, n_segments, control_points):  
#     t_space = np.linspace(0, 1, n_segments + 1)
#     for i, dt in enumerate(t_space):
#         p0 = bz.linear(*control_points[0:2], dt)
#         p1 = bz.linear(*control_points[1:3], dt)
#         d.add_line(figure, p0, p1, i)

# plot_quadratic(fig, n_segments, control_points)
# fig.update_traces(connectgaps=False)
# fig.update_layout(title="Linear Interpolations")
# fig.show("png")

In [17]:
control_points = np.array([
    [0, 0],
    [0.5, 0],
    [0.5, 1],
    [1, 1]
], dtype=float)

In [18]:
# fig.data = []

# d.add_control_points(fig, control_points)

# fig.update_layout(title="Control Points")
# fig.show("png")

In [19]:
n_segments = 10
curve = bz.get_curve(n_segments, control_points)
print("curve", curve.shape)

curve (11, 2)


In [20]:
# d.add_curve(fig, curve)

# fig.update_layout(title="Cubic Bezier Curve")
# fig.show("png")

In [21]:
# fig.data = []

# def plot_cubic(figure, n_segments, control_points):
#     t_space = np.linspace(0, 1, n_segments + 1)

#     for i, dt in enumerate(t_space):
#         p0 = bz.quadratic(*control_points[0:3], dt)
#         p1 = bz.quadratic(*control_points[1:4], dt)
#         d.add_line(fig, p0, p1, i)

# plot_cubic(fig, n_segments, control_points)

# fig.update_traces(connectgaps=False)
# fig.update_layout(title="Linear Interpolations")
# fig.show("png")

In [22]:
# fig.data = []

# plot_quadratic(fig, n_segments, control_points[0:3])
# plot_quadratic(fig, n_segments, control_points[1:4])

# fig.update_traces(connectgaps=False)
# fig.update_layout(title="Linear Interpolations")
# fig.show("png")

In [23]:
control_points = np.array([
    [0, 0],
    [0, 1],
    [0.5, 1],
    [0.5, 0],
    [1, 0],
    [1, 1]
], dtype=float)

In [24]:
# fig.data = []

# d.add_control_points(fig, control_points)

# fig.update_layout(title="Control Points")
# fig.show("png")

In [25]:
n_segments = 25
curve = bz.get_curve(n_segments, control_points)
print("curve", curve.shape)

curve (26, 2)


In [26]:
# d.add_curve(fig, curve)

# fig.update_layout(title="Bezier Curve")
# fig.show("png")